<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data. 

Some familiarity with Python is helpful. This notebook is compatible with Python 3.6.

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

## Learning goals

In this notebook you learn how to:

-  Set up a Watson Machine Learning instance.
-  Deploy the SPSS model online.
-  Use the deployed model to score data.


## Contents

1.	[Set up the environment](#setup)
2.	[Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository](#scoring)
3.	[Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">here</a>).

-   Install <a href="https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search" target="_blank" rel="noopener noreferrer">watson-machine-learning-client</a> package (not required on Watson Studio).

#### Install the `watson-machine-learning-client` package from pypi.  
**Note:** `watson-machine-learning-client` documentation can be found <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install --upgrade watson-machine-learning-client 

Authenticate the Watson Machine Learning service on the IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener no referrer">Service Credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [3]:
wml_credentials = {
    "apikey": "...",
    "username": "...",
    "password": "...",
    "instance_id": "...",
    "url": "https://ibm-watson-ml.mybluemix.net"
}

Import the `watson-machine-learning-client` module and authenticate the service instance.

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

You cand check the details of your Watson Machine Learning service by calling the ``service_instance.details`` attribute.

In [7]:
instance_details = client.service_instance.details

Get a list of stored models in the WML repository.

In [8]:
client.repository.list_models()

------------------------------------  -------------------------------------------  ------------------------  -----------------
GUID                                  NAME                                         CREATED                   FRAMEWORK
53b90fb3-0d12-4d66-96c8-b4e4cf96a0ae  Customer churn Spark model                   2019-07-03T15:47:49.776Z  mllib-2.3
ad24c140-f97a-49f2-b02c-f8ce44a58c27  Custom ARIMA estimator for sklearn pipeline  2019-07-03T01:04:33.998Z  scikit-learn-0.19
471b39cc-9c3b-4ff9-a8ea-0297efe0ca5d  Boston house price prediction                2019-05-20T18:19:55.433Z  scikit-learn-0.19
fc5462c8-7eb9-4dda-8b0a-947e2faa30da  WML Product Line Prediction Model            2019-05-17T17:33:53.278Z  mllib-2.3
a57e82a9-076e-4236-8bd2-7465e726c419  WML Product Line Prediction Model            2019-05-17T17:28:36.434Z  mllib-2.3
914b598b-f52e-4a87-bfed-2f4163eba25e  Boston house price prediction                2019-05-13T21:11:04.132Z  scikit-learn-0.19
43dbda1f-34f4-43

Get a list of deployed models.

In [9]:
client.deployments.list()

------------------------------------  -------------------------------------------------------  -------  ------------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                     TYPE     STATE               CREATED                   FRAMEWORK          ARTIFACT TYPE
d0b1dadf-c0f7-4471-b8d2-08f9a108ce1c  Web scraping python function deployment                  online   DEPLOY_SUCCESS      2019-07-08T22:01:51.009Z  n/a                function
7985e484-4192-400e-82e5-3756d4600668  ARIMA model python function deployment                   online   DEPLOY_SUCCESS      2019-07-03T20:33:40.817Z  n/a                function
6aedb5b7-638a-4388-ab0d-45fecb3b7081  Customer Churn Prediction                                online   DEPLOY_SUCCESS      2019-07-03T15:51:34.166Z  mllib-2.3          model
cfcd5f9e-5b07-4bea-b57d-304c12254add  sklearn_pipeline_arima                                   online  

<a id="scoring"></a>
## 2. Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository

In this section, you will learn how to:
- [2.1 Download and save an SPSS model](#save)
- [2.2 Create an online deployment](#create)
- [2.3 Score a test data record](#score)

For more information about REST APIs, please see the <a href="http://watson-ml-api.mybluemix.net/" target="_blank" rel="noopener no referrer">Swagger Documentation</a>.

### 2.1 Download and save an SPSS model<a id="save"></a>

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

**Note:** You may need to install the `wget` package. To install the `wget` package, run the following command. 

In [ ]:
!pip install --upgrade wget

In [11]:
# Download the sample SPSS model.
import os
import wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the SPSS sample model in your WML repository.

In [12]:
# Save the SPSS model to the WML repository.
props_spss = {
    client.repository.ModelMetaNames.NAME: 'SPSS model for Churn prediction',
    client.repository.ModelMetaNames.AUTHOR_NAME: 'IBM', 
    client.repository.ModelMetaNames.FRAMEWORK_NAME: 'spss-modeler', 
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: '18.1'
}

model_details = client.repository.store_model(filename, props_spss)

Get details of all the models saved in the repository.

In [13]:
# List models in the repository.
print(model_details)

{'metadata': {'guid': 'a3805071-7c9f-4381-b18d-ec97657f6669', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/a3805071-7c9f-4381-b18d-ec97657f6669', 'created_at': '2019-07-08T22:54:19.037Z', 'modified_at': '2019-07-08T22:54:19.096Z'}, 'entity': {'runtime_environment': 'None Provided', 'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/a3805071-7c9f-4381-b18d-ec97657f6669/learning_configuration', 'author': {'name': 'IBM'}, 'name': 'SPSS model for Churn prediction', 'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/a3805071-7c9f-4381-b18d-ec97657f6669/learning_iterations', 'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/a3805071-7c9f-4381-b18d-ec97657f6669/feedback', 'latest_version': {

### 2.2 Create an online deployment<a id="create"></a>

Using the code below to create an online deployment.

In [14]:
model_uid = client.repository.get_model_uid(model_details)
model_deployment = client.deployments.create(model_uid, 'Sample SPSS model deployment')



#######################################################################################

Synchronous deployment creation for uid: 'a3805071-7c9f-4381-b18d-ec97657f6669' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS........
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='442fd8a0-5d70-4ce1-a871-dfd430b70de3'
------------------------------------------------------------------------------------------------




Now, you can check details of your deployments.

In [15]:
deployment_uid = client.deployments.get_uid(model_deployment)
print(client.deployments.get_details(deployment_uid))

{'metadata': {'guid': '442fd8a0-5d70-4ce1-a871-dfd430b70de3', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/deployments/442fd8a0-5d70-4ce1-a871-dfd430b70de3', 'created_at': '2019-07-08T22:54:21.793Z', 'modified_at': '2019-07-08T22:54:22.058Z'}, 'entity': {'runtime_environment': 'None Provided', 'name': 'Sample SPSS model deployment', 'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/deployments/442fd8a0-5d70-4ce1-a871-dfd430b70de3/online', 'deployable_asset': {'name': 'SPSS model for Churn prediction', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/a3805071-7c9f-4381-b18d-ec97657f6669', 'guid': 'a3805071-7c9f-4381-b18d-ec97657f6669', 'created_at': '2019-07-08T22:54:21.762Z', 'type': 'model'}, 'description': 'Description of deployment', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'model_type': 'spss-modeler-18.1', '

### 2.3 Score a test data record<a id="score"></a>

Get the scoring endpoint, `scoring_url`, for your deployed model using the code below.

In [ ]:
# Get the scoring endpoint.
scoring_url = client.deployments.get_scoring_url(model_deployment)
print(scoring_url)

Use the client `score` method to score test data about a customer.

In [17]:
scoring_payload = {'fields':['customerID','gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges','TotalCharges','Churn','SampleWeight'],'values':[['3638-WEABW','Female',0,'Yes','No',58,'Yes','Yes','DSL','No','Yes','No','Yes','No','No','Two year','Yes','Credit card (automatic)',59.9,3505.1,'No',2.768]]}
prediction = client.deployments.score(scoring_url, scoring_payload)

print(prediction)

{'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn'], 'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}


As you can see, this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="summary"></a>
## 3. Summary     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2017-2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>